Prototyping filon single evaluation
=========

The goal is to evaluate a highly oscillatory integral with the filon method. 
Thereby one only needs to evaluate the integral on one frequency for each point in the grid.
(current build evaluates the whole frequency array at each point of the grid)

The (test) integral to evaluate reads:
$$
I(a,r) = \int_0^{2\pi} \log(ax) \cos(rx)\, dx.
$$
The exact solution is therefor:
$$
I(a,r) = \frac{1}{r}(\log(ax)\sin(rx) - \text{Si}(rx))
$$
The test case: we want to evaluate $I(a,a)$, $I(a,a^2 + 1)$ for some random a in the interval $(10^{-3},2)$.

Module: filon.py
--------

In [1]:
from numpy import sin, cos, pi, mod, \
    zeros, arange, linspace, require, \
    where, sum,isscalar

from __future__ import print_function

__doc__ = """
Python implementation of Filon's integration formula.

For information about Filon's formula, see e.g.
Abramowitz, Stegun, Handbook of Mathematical Functions, section 25,
http://mathworld.wolfram.com/FilonsIntegrationFormula.html,
Allen, Tildesley, Computer Simulation of Liquids, Appendix D.

Integration is performed along one dimension (default axis=0), e.g.

 [F0[0]  F1[0] ..  FN[0] ]     [f0[0]  f1[0] ..  fN[0] ]
 [   .      .         .  ]     [   .      .         .  ]
 [F0[.]  F1[.] ..  FN[.] ] = I([f0[.]  f1[.] ..  fN[.] ], dx, [k[0] .. k[Nk]])
 [   .      .         .  ]     [   .      .         .  ]
 [F0[Nk] F1[Nk] .. FN[Nk]]     [f0[Nx] f1[Nx] .. fN[Nx]]

where k and Fj have end index Nk, and fj have end index Nx.
Nk is arbitray (implicitly derived from the length of k).
Due to the algorithm, fj[Nx] must be of odd length (Nx must be an even number),
and should correspond to a linearly spaced set of data points (separated by
dx along the integration axis).

sin_integral and cos_integral allows for shifted
integration intervals by the optional argument x0.

"""


def fourier_cos(f, dx, k=None, axis=0):
    """Calculate a direct fourier cosine transform of function f(x) using
    Filon's integration method

    k, F = fourier_cos(f, dx)
    Array values f[0]..f[2n] is expected to correspond to f(0.0)..f(2n*dx),
    hence, it should contain an odd number of elements.
    The transform is approximated with the integral
    2*\int_{0}^{xmax}
    where xmax = 2n*dx

    If k is not provided, linspace(0.0, 2*pi/dx, f.shape[axis]),
    will be used.
    """

    if k is None:
        k = linspace(0.0, 2*pi/dx, f.shape[axis])

    return k, 2*cos_integral(f, dx, k, x0=0.0, axis=axis)


def cos_integral(f, dx, k, x0=0.0, axis=0):
    """\int_{x0}^{2n*dx} f(x)*cos(k x) dx

    f must have length 2n+1 along the integration axis.
    """
    return _gen_sc_int(f, dx, k, x0, axis, cos)

def sin_integral(f, dx, k, x0=0.0, axis=0):
    """\int_{x0}^{2n*dx} f(x)*sin(k x) dx

    f must have length 2n+1 along the integration axis.
    """
    return _gen_sc_int(f, dx, k, x0, axis, sin)

def _gen_sc_int(f, dx, k, x0, axis, sc):

    f = require(f)
    print("f.shape:",f.shape)
    print("f.ndim:",f.ndim)
    k = require(k)
    print('k.shape',k.shape)

    try:
        axis = range(f.ndim)[axis]
    except (IndexError, TypeError):
        print('Error: axis(=%s) is invalid' % str(axis))
        raise

    if k.ndim != 1:
        raise ValueError('k is not one dimensional, it is: \n%s \n%s \n%s \nisscalar: %s'%(type(k),k.ndim,k,isscalar(k)))

    Nk = len(k)
    print('Nk',Nk)
    Nx = f.shape[axis]
    print('Nx',Nx)
    x_shape = [1]*f.ndim + [Nx] # We'll transpose axis and put it last
    print('x_shape',x_shape)
    k_shape = [1]*(f.ndim+1)    #
    k_shape[axis] = Nk          #
    print('k_shape',k_shape)

    if mod((Nx-1), 2) != 0 or Nx < 3:
        raise ValueError('f must have an odd length, >=3, along its integration axis')

    print('f.ndim',f.ndim)
    s = (slice(None),)*f.ndim
    print("s",s)
    odd_index =   s + (slice(1,None,2),)
    print('odd_index',odd_index)
    even_index =  s + (slice(0,None,2),)
    print('even_index',even_index)
    first_index = s + ((0,),)
    print('first_index',first_index)
    last_index =  s + ((-1,),)
    print('last_index',last_index)

    alpha, beta, gamma = [x.reshape(k_shape[:-1]) for x in _alpha_beta_gamma(dx*k)]
    print('alpha.shape',alpha.shape)
    print('beta.shape',beta.shape)
    print('gamma.shape',gamma.shape)
    x = (x0+dx*arange(0.0, Nx)).reshape(x_shape)
    print("x.shape",x.shape)
    k = k.reshape(k_shape)
    print('k_shape',k_shape)

    # Add an extra dimension to f, and transpose it to put the x-dimension at axis=-1
    t = range(f.ndim+1)
    t[axis], t[-1] = t[-1], t[axis]
    f = f.reshape(f.shape+(1,)).transpose(t)
    print("f.shape:",f.shape)
    sc_k_x = sc(k*x)
    sc_k_x[first_index] *= 0.5
    sc_k_x[last_index] *= 0.5
    
    print('sc_k_x.shape',sc_k_x.shape)
    print('sc_k_x[even_index].shape',sc_k_x[even_index].shape)
    
    if sc == sin:
        return dx*(alpha * sum((f[first_index] * cos(k*x0) -
                                f[last_index]  * cos(k*x[last_index])), axis=-1) +
                   beta  * sum(f[even_index] * sc_k_x[even_index],      axis=-1) +
                   gamma * sum(f[odd_index]  * sc_k_x[odd_index],       axis=-1))

    elif sc == cos:
        return dx*(alpha * sum((f[last_index]  * sin(k*x[last_index]) -
                                f[first_index] * sin(k*x0)),         axis=-1) +
                   beta  * sum(f[even_index] * sc_k_x[even_index],   axis=-1) +
                   gamma * sum(f[odd_index]  * sc_k_x[odd_index],    axis=-1))

    raise RuntimeError('Internal error, this should not happen')



def _alpha_beta_gamma(theta):
    # From theta, calculate alpha, beta, and gamma

    N = len(theta)
    alpha = zeros(N)
    beta = zeros(N)
    gamma = zeros(N)

    # theta==0 needs special treatment
    I_nz, = theta.nonzero()
    I_z, = where(theta==0.0)
    if len(I_z) > 0:
        beta[I_z] = 2.0/3.0
        gamma[I_z] = 4.0/3.0
        theta = theta[I_nz]

    sin_t = sin(theta)
    cos_t = cos(theta)
    sin2_t = sin_t*sin_t
    cos2_t = cos_t*cos_t
    theta2 = theta*theta
    itheta3 = 1.0/(theta2*theta)

    alpha[I_nz] = itheta3*(theta2 + theta*sin_t*cos_t - 2*sin2_t)
    beta[I_nz] = 2*itheta3*(theta*(1+cos2_t) - 2*sin_t*cos_t)
    gamma[I_nz] = 4*itheta3*(sin_t - theta*cos_t)

    return (alpha, beta, gamma)



In [2]:
import numpy as np
from scipy.special import sici
import time
times = {}

/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/special/__init__.py:640: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ufuncs import *
/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/linalg/basic.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._solve_toeplitz import levinson
/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/linalg/__init__.py:207: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._decomp_update import *
/Users/szabo/pyTableTest/venv/lib/python2.7/site-packages/scipy/special/_ellip_harm.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ellip_harm_2 import _ellipsoid, _ellipsoid_norm


In [38]:
# prepare initial params
aArr = np.random.uniform(1e-3,2,10)
rArr = aArr**2 + 1

print('aArr.shape',aArr.shape)
print('rArr.shape',rArr.shape) 

aArr.shape (10,)
rArr.shape (10,)


In [39]:
def exactINDEF(r,a,x):
    return (np.log(a*x)*np.sin(r*x) - sici(r*x)[0])/r

def exact(r,a,low,up):
    return exactINDEF(r,a,up) - exactINDEF(r,a,low)

start = time.time()
resEall = exact(rArr,aArr,1,2.0*np.pi)
end = time.time() - start
times['exactAll'] = end

print('resEall.shape',resEall.shape)

resEall.shape (10,)


In [40]:

def integrand(a,x):
    return np.log(a*x)

def filonIntAll(r,a,low,up,n=1001): 
    aGrid = a[np.newaxis,:]
    args,steps = np.linspace(low,up,n,retstep=True,endpoint=True)
    argsGrid = args[:,np.newaxis]
    intArr = integrand(aGrid,argsGrid)
    res = cos_integral(intArr,steps,r,low,axis=0)
    #print a.shape
    return res

start = time.time()
resFraw = filonIntAll(rArr,aArr,1,2.0*np.pi)
resFall = np.diagonal(resFraw)
end = time.time() - start
times['filonAll'] = end


print('resFall.shape',resFall.shape)

f.shape: (1001, 10)
f.ndim: 2
k.shape (10,)
Nk 10
Nx 1001
x_shape [1, 1, 1001]
k_shape [10, 1, 1]
f.ndim 2
s (slice(None, None, None), slice(None, None, None))
odd_index (slice(None, None, None), slice(None, None, None), slice(1, None, 2))
even_index (slice(None, None, None), slice(None, None, None), slice(0, None, 2))
first_index (slice(None, None, None), slice(None, None, None), (0,))
last_index (slice(None, None, None), slice(None, None, None), (-1,))
alpha.shape (10, 1)
beta.shape (10, 1)
gamma.shape (10, 1)
x.shape (1, 1, 1001)
k_shape [10, 1, 1]
f.shape: (1, 10, 1001)
sc_k_x.shape (10, 1, 1001)
sc_k_x[even_index].shape (10, 1, 501)
resFall.shape (10,)


In [41]:
#ind = 0
#for ir,rel in enumerate(rArr):
#    for ia,ael in enumerate(aArr):
#        print "%d:\t %1.4e\t%1.4e\t| %1.4e \t%1.4e \t| %1.4e \t%1.4e"%(ind,rel,ael,resFall[ir][ia],resEall[ir][ia],resFall[ir][ia]-resEall[ir][ia],np.abs((resFall[ir][ia]-resEall[ir][ia])/(resFall[ir][ia]+resEall[ir][ia])))
#       ind+=1
calls = len(rArr)
print("full time (%d calls):  \t%1.4e | %1.4e"%(calls,times['filonAll'],times['exactAll']))
print("time per call:         \t\t%1.4e | %1.4e"%(times['filonAll']/calls,times['exactAll']/calls))

full time (10 calls):  	9.9730e-03 | 2.5201e-04
time per call:         		9.9730e-04 | 2.5201e-05


In [42]:
print(resFall)
print(resEall)



relErr = np.abs((resFall-resEall)/(resFall+resEall))



i = np.unravel_index(relErr.argmax(), relErr.shape)
print("max. relErr: (%1.4e,%1.4e) %1.4e"%(rArr[i],aArr[i],relErr[i]))

[ 0.17053376 -0.11987018  0.22058861  0.79840037  0.03652049 -0.39017931
  0.97565522 -0.06118498 -0.23911501 -0.70434216]
[ 0.17053376 -0.11987018  0.22058861  0.79840037  0.03652049 -0.39017931
  0.97565522 -0.06118498 -0.23911501 -0.70434216]
max. relErr: (1.4939e+00,7.0278e-01) 1.5666e-10


In [ ]:
s = (slice(0,None,2))

In [ ]:
print(s)

In [ ]:
a=linspace(0,10,100)
print(a)

In [ ]:
print(a[s])